### 0. Load environment variables
#### Case 1) If you use OpenAI's ChatGPT, then you need store the following variables in '.env' file:

- 'OPENAI_API_KEY' = XXXXXXXXXXXXXXX
- 'model_name' = XXXXXXXXXXXXXXX

##### Case 2) If you use AzureOpenAI's ChatGPT, then you need store the following variables in '.env' file:

- 'OPENAI_API_TYPE' = XXXXXXXXXXXXXXX
- 'OPENAI_API_VERSION' = XXXXXXXXXXXXXXX
- 'OPENAI_API_BASE' = XXXXXXXXXXXXXXX
- 'OPENAI_API_KEY' = XXXXXXXXXXXXXXX
- 'deployment_name' = XXXXXXXXXXXXXXX

In [1]:
import os
import sys  
from dotenv import load_dotenv
sys.path.extend(["..", '../..'])  
_ = load_dotenv('.env')


model_name = os.getenv('deployment_name') if os.getenv('OPENAI_API_TYPE') == 'azure' else os.getenv('model_name')
if os.getenv('OPENAI_API_TYPE') == 'azure':
    deployment_name = model_name

### 1. Get a random persona

In [2]:
from utils.pydantic_models.user_persona import UserPersona, get_random_user_persona
persona = get_random_user_persona().json()
persona

'{"gender": "Non_binary", "name": "Benjamin Barker", "language": "English", "nationality": "Thailand", "age": 88, "hobbies": ["DIY_projects", "sports", "hiking", "cooking"], "talkative": true, "characteristics": ["NotImpulsive", "NotArrogant"], "education_level": "unknown"}'

### 2. Fetch the prompt and create a chain (using LCEL)

In [3]:
from langchain import hub
from langchain.chat_models import AzureChatOpenAI, ChatOpenAI

prompt = hub.pull("jet-taekyo-lee/persona-based-dialog")
deployment_name = os.getenv('deployment_name')

chat_model = AzureChatOpenAI(deployment_name=deployment_name, temperature=0) if os.getenv('OPENAI_API_TYPE') == 'azure' else ChatOpenAI(model_name=model_name, temperature=0)

def streaming_output(response_generator):
    for chunk in response_generator:
        print(chunk.content, end='')    

chain = prompt | chat_model

### 3. Get a response with an empty input (Initiating a conversation)

In [4]:
response = chain.stream({'persona':persona, 'input': ''})
streaming_output(response)

## Persona Analysis
- **Characteristics**: NotImpulsive, NotArrogant
- **Other Attributes**: Non-binary, named Benjamin Barker, speaks English, from Thailand, 88 years old, enjoys DIY projects, sports, hiking, and cooking, talkative, education level unknown.

## Conflict Detection in persona
- **Conflict 1**: The name 'Benjamin Barker' is not typically Thai, which conflicts with the nationality of 'Thailand'.
- **Conflict 2**: The age of 88 and the hobbies of sports and hiking might conflict as these activities are typically associated with younger, more physically able individuals.

## Conflict Resolution in persona
- **Adjusted Characteristics**: The name can be adjusted to 'Benjawan Barker', a more Thai-sounding first name while keeping the last name. The age and hobbies conflict can be resolved by assuming that Benjamin is in excellent health for their age and enjoys less strenuous sports and hiking activities.

## Detailed Persona Description
Benjawan Barker is a non-binary indivi

In [5]:
response = chain.stream({'persona':persona, 'input': 'Life is so boring. Why is that?'})
streaming_output(response)

## Persona Analysis
- **Characteristics**: NotImpulsive, NotArrogant
- **Other Attributes**: Non-binary, named Benjamin Barker, speaks English, Thai nationality, 88 years old, enjoys DIY projects, sports, hiking, and cooking, talkative, education level unknown.

## Conflict Detection in persona
- **Conflict 1**: The name 'Benjamin Barker' is not typically Thai. This could be a potential conflict with the nationality stated as 'Thailand'.
- **Conflict 2**: The age of 88 might conflict with the hobbies listed, such as sports and hiking, which are typically more physically demanding activities.

## Conflict Resolution in persona
- **Adjusted Characteristics**: The name can be interpreted as Benjamin Barker being of mixed heritage or having lived in an English-speaking country for a significant part of their life. As for the age and hobbies, Benjamin Barker could be a very fit and active 88-year-old, or they could enjoy less strenuous sports and hiking trails.

## Detailed Persona Descript